# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [32]:
# Step 1: Create your prompts
import requests
import ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

#create message
messages = [
    {"role": "user", "content": 
     """
   Fixing Failed to retrieve data for TaxID 3502447502: status code 403 in the below python code. This may be because url does not have headers
   
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from time import sleep

class Website:
    def __init__(self, base_url):
        self.base_url = base_url
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
        }

# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty dictionary to store the scraped data
scraped_data = {}

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base_url + str(tax_id)
    
    # Send a GET request to the updated URL
    response = requests.get(updated_url, timeout=5)
    
    # If the GET request is successful, scrape the data
    if response.status_code == 200:
        # Get the content of the page
        page_content = response.content
        
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(page_content, 'html.parser')
        
        # Find all elements on the page that contain text
        for element in soup.find_all(text=True):
            # Remove leading/trailing whitespace from the text
            text = element.strip()
            
            # If the text is not empty, add it to the scraped data dictionary
            if text:
                scraped_data[text] = updated_url
                
    else:
        print(f"Failed to retrieve data for TaxID {tax_id}: status code {response.status_code}")
        
    # Add a delay to avoid overwhelming the server
    sleep(1)

# Print the scraped data
for key, value in scraped_data.items():
    print(f"{key}: {value}")
     """
    }
]
#create payload
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

#print the result
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])




The issue you're encountering is indeed due to the lack of headers in your request. Here's an updated version of your code with headers included:

```python
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from time import sleep

class Website:
    def __init__(self, base_url):
        self.base_url = base_url
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
        }

# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty dictionary to store the scraped data
scraped_data = {}

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base

In [15]:
# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty list to store the updated URLs
updated_urls = []

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base_url + str(tax_id)
    updated_urls.append(updated_url)

# Print the updated URLs
print("Updated URLs:")
for url in updated_urls:
    print(url)

Updated URLs:
https://doanhnghiep.biz/3502447502
https://doanhnghiep.biz/030352220


In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from time import sleep

# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty dictionary to store the scraped data
scraped_data = {}

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base_url + str(tax_id)
    
    # Send a GET request to the updated URL
    response = requests.get(updated_url, timeout=5)
    
    # If the GET request is successful, scrape the data
    if response.status_code == 200:
        # Get the content of the page
        page_content = response.content
        
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(page_content, 'html.parser')
        
        # Find all elements on the page that contain text
        for element in soup.find_all(text=True):
            # Remove leading/trailing whitespace from the text
            text = element.strip()
            
            # If the text is not empty, add it to the scraped data dictionary
            if text:
                scraped_data[text] = updated_url
                
    else:
        print(f"Failed to retrieve data for TaxID {tax_id}: status code {response.status_code}")
        
    # Add a delay to avoid overwhelming the server
    sleep(1)

# Print the scraped data
for key, value in scraped_data.items():
    print(f"{key}: {value}")

Failed to retrieve data for TaxID 3502447502: status code 403
Failed to retrieve data for TaxID 030352220: status code 403


In [33]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from time import sleep

class Website:
    def __init__(self, base_url):
        self.base_url = base_url
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
        }

# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty dictionary to store the scraped data
scraped_data = {}

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base_url + str(tax_id)
    
    # Send a GET request to the updated URL with headers
    response = requests.get(updated_url, headers=self.headers, timeout=5)
    
    # If the GET request is successful, scrape the data
    if response.status_code == 200:
        # Get the content of the page
        page_content = response.content
        
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(page_content, 'html.parser')
        
        # Find all elements on the page that contain text
        for element in soup.find_all(text=True):
            # Remove leading/trailing whitespace from the text
            text = element.strip()
            
            # If the text is not empty, add it to the scraped data dictionary
            if text:
                scraped_data[text] = updated_url
                
    else:
        print(f"Failed to retrieve data for TaxID {tax_id}: status code {response.status_code}")
        
    # Add a delay to avoid overwhelming the server
    sleep(1)

# Print the scraped data
for key, value in scraped_data.items():
    print(f"{key}: {value}")

NameError: name 'self' is not defined

In [ ]:
"""
     create a python program to adding website + TaxID   
I have a list of TaxID as below     
List TaxID = 5300829397, 5300773641

The website is https://doanhnghiep.biz/

Example result is added to url list
https://doanhnghiep.biz/5300829397
https://doanhnghiep.biz/5300773641
     """

